### Dataset:
Fisher, R. (1936). Iris [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C56C76.

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,normalize
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf

### Preprocessing

In [4]:
cols = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
df = pd.read_csv('iris.data', names=cols)
df['class'] = df['class'].map({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

def scale_dataset(dataframe):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  data = np.hstack((X, np.reshape(y, (-1, 1))))

  return data, X, y

train, X_train, y_train = scale_dataset(train)
val, X_val, y_val = scale_dataset(val)
test, X_test, y_test = scale_dataset(test)
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Neural Network

In [5]:
def train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
  nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(num_nodes, activation="sigmoid", input_shape=(4,)),
    tf.keras.layers.Dropout(dropout_prob),
    tf.keras.layers.Dense(num_nodes, activation="sigmoid"),
    tf.keras.layers.Dropout(dropout_prob),
    tf.keras.layers.Dense(3, activation="softmax")
  ])

  nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='categorical_crossentropy',
                   metrics=['accuracy'])
  history = nn_model.fit(
      X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0
  )
  return history, nn_model

In [6]:
epochs = 10
least_val_loss = float('inf')
least_loss_model = None
for num_nodes in [16, 32, 64]:
  for dropout_prob in [0, 0.2]:
    for lr in [0.01, 0.005, 0.001]:
      for batch_size in [32, 64, 128]:
        history, model = train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs)
        val_loss = model.evaluate(X_val, y_val)[0]
        print(f"{num_nodes} nodes, dropout {dropout_prob}, lr {lr}, batch size {batch_size}, val loss {val_loss}")
        if val_loss < least_val_loss:
          least_val_loss = val_loss
          least_loss_model = model

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9333 - loss: 0.8554
16 nodes, dropout 0, lr 0.01, batch size 32, val loss 0.8553673028945923
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7333 - loss: 0.8843
16 nodes, dropout 0, lr 0.01, batch size 64, val loss 0.8843175172805786
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6000 - loss: 0.9853
16 nodes, dropout 0, lr 0.01, batch size 128, val loss 0.985338568687439
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6333 - loss: 0.9347
16 nodes, dropout 0, lr 0.005, batch size 32, val loss 0.934740424156189
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5333 - loss: 1.0216
16 nodes, dropout 0, lr 0.005, batch size 64, val loss 1.0215505361557007
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3000 - loss: 1.1689
16 nodes, dropout 0, lr 0.005, batch size 128, val loss 1.1689298152923584
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3000 - loss: 1.1110
16 nodes, dropout 0, lr 0.001, batch size 32, val loss 

In [29]:
y_pred = least_loss_model.predict(X_test)
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.80      0.80      0.80        10
           2       0.89      0.80      0.84        10

   micro avg       0.90      0.87      0.88        30
   macro avg       0.90      0.87      0.88        30
weighted avg       0.90      0.87      0.88        30
 samples avg       0.87      0.87      0.87        30



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
